# Testing a Custom Redis Server with `redis-py`

This notebook provides an interactive way to test a custom, Redis-compatible server using the standard `redis-py` Python client.

### Instructions:
1. Make sure your custom Zig-based Redis server is running.
2. Run the cells sequentially by pressing `Shift + Enter`.
3. Observe the output of each cell to verify that your server responds correctly.

In [1]:
# Step 1: Install and import necessary libraries
!pip install redis
import redis
import time

In [2]:
# Step 2: Configure the connection to your server
REDIS_HOST = "localhost"
REDIS_PORT = 8080

In [3]:
# Step 3: Create the client and test the connection
try:
    # decode_responses=True makes the client return Python strings instead of bytes.
    r = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, decode_responses=True)
    
    # The .ping() command is a great way to check if the connection is alive.
    is_connected = r.ping()
    print(f"Successfully connected to server: {is_connected}")
except (redis.exceptions.ConnectionError, redis.exceptions.TimeoutError) as e:
    print(f"Failed to connect to {REDIS_HOST}:{REDIS_PORT}. Please ensure the server is running.")
    print(f"Error: {e}")

Successfully connected to server: True


--- 
## Command Tests
Now we will run tests for each command.

In [4]:
# Before starting, let's clear the database to ensure a clean state.
print("Running FLUSHDB to clear all keys...")
try:
    r.flushdb()
    print("Database flushed successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Running FLUSHDB to clear all keys...
Database flushed successfully.


In [5]:
# Test: PING
# Checks if the server is responsive.
print(f"PING response: {r.ping()}")

PING response: True


In [6]:
# Test: ECHO
# The server should return the exact string sent.
message = "Hello from Jupyter!"
response = r.echo(message)
print(f"Sent: '{message}'")
print(f"Received: '{response}'")

Sent: 'Hello from Jupyter!'
Received: 'Hello from Jupyter!'


In [7]:
# Test: SET and GET
# Set a key to a value, then retrieve it.
key, value = "my_key", "my_value"
set_response = r.set(key, value)
print(f"SET '{key}' to '{value}'. Server response: {set_response}")

get_response = r.get(key)
print(f"GET '{key}'. Server response: '{get_response}'")

# Test getting a key that does not exist
non_existent_key = "missing_key"
get_none_response = r.get(non_existent_key)
print(f"GET '{non_existent_key}'. Server response: {get_none_response} (should be None)")

SET 'my_key' to 'my_value'. Server response: True
GET 'my_key'. Server response: 'my_value'
GET 'missing_key'. Server response: None (should be None)


In [8]:
# Test: EXISTS and DEL
# Check if a key exists, then delete it.
key_to_delete = "temp_key"
r.set(key_to_delete, "some value")
print(f"Set key '{key_to_delete}'.")

exists_response = r.exists(key_to_delete)
print(f"EXISTS '{key_to_delete}'. Server response: {exists_response} (1 means true)")

del_response = r.delete(key_to_delete)
print(f"DEL '{key_to_delete}'. Server response: {del_response} (1 means one key was deleted)")

exists_after_del_response = r.exists(key_to_delete)
print(f"EXISTS '{key_to_delete}' again. Server response: {exists_after_del_response} (0 means false)")

Set key 'temp_key'.
EXISTS 'temp_key'. Server response: 1 (1 means true)
DEL 'temp_key'. Server response: 1 (1 means one key was deleted)
EXISTS 'temp_key' again. Server response: 0 (0 means false)


In [9]:
# Test: INCR, DECR, INCRBY, DECRBY
# Atomic integer operations.
counter_key = "my_counter"
r.set(counter_key, 10)
print(f"Set '{counter_key}' to 10.")

incr_val = r.incr(counter_key)
print(f"INCR '{counter_key}'. New value: {incr_val}")

decr_val = r.decr(counter_key)
print(f"DECR '{counter_key}'. New value: {decr_val}")

incrby_val = r.incrby(counter_key, 5)
print(f"INCRBY '{counter_key}' by 5. New value: {incrby_val}")

decrby_val = r.decrby(counter_key, 3)
print(f"DECRBY '{counter_key}' by 3. New value: {decrby_val}")

Set 'my_counter' to 10.
INCR 'my_counter'. New value: 11
DECR 'my_counter'. New value: 10
INCRBY 'my_counter' by 5. New value: 15
DECRBY 'my_counter' by 3. New value: 12


In [10]:
# Test: HSET, HGET, HGETALL
# Hash operations for storing objects.
hash_key = "user:101"
print(f"Working with hash key: '{hash_key}'")

hset_resp1 = r.hset(hash_key, "name", "Bob")
print(f"HSET name=Bob. Fields added: {hset_resp1}")

hset_resp2 = r.hset(hash_key, "email", "bob@example.com")
print(f"HSET email=bob@example.com. Fields added: {hset_resp2}")

hget_resp = r.hget(hash_key, "name")
print(f"HGET name. Value: '{hget_resp}'")

hgetall_resp = r.hgetall(hash_key)
print(f"HGETALL. Value: {hgetall_resp}")

Working with hash key: 'user:101'
HSET name=Bob. Fields added: 1
HSET email=bob@example.com. Fields added: 1
HGET name. Value: 'Bob'
HGETALL. Value: {'name': 'Bob', 'email': 'bob@example.com'}


In [11]:
# Test: TYPE
# Checks the data type of a key.
str_key = "a_string_key"
hash_key = "a_hash_key"
none_key = "a_none_key"

r.set(str_key, "hello")
r.hset(hash_key, "f1", "v1")

print(f"TYPE of '{str_key}': {r.type(str_key)}")
print(f"TYPE of '{hash_key}': {r.type(hash_key)}")
print(f"TYPE of '{none_key}': {r.type(none_key)}")

TYPE of 'a_string_key': string
TYPE of 'a_hash_key': hash
TYPE of 'a_none_key': none


In [12]:
# Test: GETDEL
# Atomically gets a key's value and then deletes the key.
getdel_key = "key_to_getdel"
r.set(getdel_key, "this will disappear")
print(f"Set '{getdel_key}' to 'this will disappear'")

# We must use execute_command because GETDEL is not a high-level function in redis-py
getdel_val = r.execute_command("GETDEL", getdel_key)
print(f"GETDEL '{getdel_key}'. Value returned: '{getdel_val}'")

exists_val = r.exists(getdel_key)
print(f"EXISTS '{getdel_key}' after GETDEL. Response: {exists_val} (should be 0)")

Set 'key_to_getdel' to 'this will disappear'
GETDEL 'key_to_getdel'. Value returned: 'this will disappear'
EXISTS 'key_to_getdel' after GETDEL. Response: 0 (should be 0)
